# Collective progress

**Example notebook for creating anonymised, collective information on progress**

* Before running this notebook, you need to prepare the data you want to assess. To do so, please use the notebooks  "prepare-PRIMAP-hist-data-for-collective-progress-plots.ipynb" or "prepare-PRIMAP-data.ipynb" in case the data you need is not yet available.
* For testing, some example data is available in the folder "proc_data"
* Enter the name of the file that you wish to use in the second cell and some of the plotting parameters. After that you can run the full notebook with minimal changes. 

In [1]:
# import modules

# system 
import re
import os

# calculation
import pandas as pd
import numpy as np

# plotting
%matplotlib inline
import seaborn
import matplotlib

# global stocktake tools
from gst_tools.make_plots import *
import gst_tools.gst_utils as utils


In [16]:
# USER INPUT

# First, choose which file you want to plot the data for
data_file_name = 'PRIMAP-hist_v2.0_KyotoGHG-AR4-total-excl-LU.csv'

# other options include...
# 'PRIMAP-hist_v2.0_KyotoGHG-AR4-total-excl-LU.csv'
# 'UN-population-data-2017.csv'
# 'PRIMAP-hist_v2.0_Energy-CO2.csv'
# 'PRIMAP-hist_UN-2017_calc__CO2-per-population.csv'
# 'PRIMAP-hist_UN-2017_calc_CO2-total-excl-LU-per-population.csv'
# 'WDI2017_GDP-PPP.csv'

# Second, choose which years you are interested in analysing
years_of_interest = ['1990', '2000', '2014']

# Third, update data description display names!
# TODO - default to automatic if not specified.
variable_name_to_display = 'CO2 emissions'
data_source_to_display = 'PRIMAP-hist v2.0'

# Save plots?
# Set the following to True if plots should be saved. 
# If False, plots will be shown on screen but not saved to a file.
save_opt = True

In [17]:
# DATA READING AND PREP

# read the data from file 
fname_in = os.path.join('proc-data', data_file_name)
data = pd.read_csv(fname_in)

# Check the data format
if not utils.verify_data_format(data):
    print('WARNING: The data is not correctly formatted! Please check before continuing!')

# extract the key information
variable = data['variable'].unique()[0]
unit = data['unit'].unique()[0]

# tidy up for next stesps
data_years = utils.set_countries_as_index(data)
data_years = data_years.dropna(axis=1, how='any')

# remove comment below to display the data
#data_years

In [18]:
# Plot 1 - make a histogram of absolute data

# how should the xaxis be labelled?
xaxlabel = variable_name_to_display

# make a plot for each year
for selected_year in years_of_interest:
    
    title = 'distribution of ' + variable_name_to_display + ' in ' + str(selected_year)
    
    make_histogram(data_years[selected_year], unit,
                   xlabel=xaxlabel, title=title, 
                   sourcename=data_source_to_display,
                   remove_outliers=True, ktuk=3,
                   save_plot=save_opt, plot_name=(variable + '-' + 'absolute' + '-' + str(selected_year)))


-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 1990, dtype: float64)
upper outliers are: 
country
AUS     423000.0
BRA     606000.0
CAN     607000.0
CHN    3600000.0
DEU    1260000.0
ESP     294000.0
FRA     554000.0
GBR     814000.0
IDN     354000.0
IND    1150000.0
IRN     382000.0
ITA     525000.0
JPN    1270000.0
KAZ     341000.0
KOR     316000.0
MEX     423000.0
NGA     302000.0
POL     477000.0
RUS    3780000.0
UKR     952000.0
USA    6510000.0
ZAF     374000.0
Name: 1990, dtype: float64
---
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2000, dtype: float64)
upper outliers are: 
country
AUS     488000.0
BRA     803000.0
CAN     736000.0
CHN    4890000.0
DEU    1060000.0
ESP     393000.0
FRA     558000.0
GBR     726000.0
IDN     525000.0
IND    1640000.0
IRN     525000.0
ITA     560000.0
JPN    1380000.0
KOR     517000.0
MEX     550000.0
POL     397000.0
RUS    2280000.0
SAU     345000.0
TUR     301000.0
UKR 

In [19]:
# Plot 2 - trends

# Calculate trends from the absolute data
# trends - % change in any given year
# rolling_trends - % annual change averaged over the specifiied number of years (here 5 is recommended)
# trends_unit - unit of the trend, here % change
trends, rolling_trends, trends_unit = utils.calculate_trends(data_years, num_years_trend=5)

# define some labels for the plots
trends_variable = 'average annual change' 
thistitle = "5-year rolling average trend in \n" + variable_name_to_display + "\nin " + str(data_years.columns[-1])

# make a plot showing the trend in the final year of available data
make_histogram(rolling_trends.iloc[:,-1], trends_unit, 
               xlabel=trends_variable,
               title=thistitle, 
               sourcename=data_source_to_display, 
               save_plot=save_opt, plot_name=(variable + '-' + 'rolling-average'))


Averaging trend over 5 years.
bins set to range(-15, 15)


In [20]:
# Plot 3 - change since year X

# run calculations - dataframe of differences in all years relative to the specified year
# the function returns both absolute and relative (%) values
df_abs_diff_1990, df_perc_diff_1990 = utils.calculate_diff_since_yearX(data_years, '1990')
df_abs_diff_2005, df_perc_diff_2005 = utils.calculate_diff_since_yearX(data_years, '2005')

# make plots

# a few selected years; difference from 1990
for selected_year in years_of_interest:
    make_histogram(df_perc_diff_1990[selected_year], "%", 
                   xlabel='change since 1990', 
                   title=('change in ' + variable_name_to_display + '\n from 1990 to ' + str(selected_year)), 
                   sourcename=data_source_to_display,
                   remove_outliers=True, ktuk=3, 
                   save_plot=save_opt, plot_name=(variable + '-' + 'change-since-1990' + '-in-' + str(selected_year)))

    
# decrease in the last year as compared to 2005 
make_histogram(df_perc_diff_2005.iloc[:,-1], '%', 
               xlabel='change since 2005', 
               title=('change in ' + variable_name_to_display + '\n from 2005 to ' + str(df_perc_diff_2005.columns[-1])), 
               sourcename=data_source_to_display,
               remove_outliers=False, 
               save_plot=save_opt, plot_name=(variable + '-' + 'change-since-2005'))

# same plot, but removing outliers to show difference...
make_histogram(df_perc_diff_2005.iloc[:,-1], '%', 
               xlabel='change since 2005', 
               title=('change in ' + variable_name_to_display + '\n from 2005 to ' + str(df_perc_diff_2005.columns[-1])), 
               sourcename=data_source_to_display,
               remove_outliers=True, 
               save_plot=save_opt, plot_name=(variable + '-' + 'change-since-2005-excl-outliers'))


Calculating difference compared to 1990
Calculating difference compared to 2005
---------
All values in the series are the same! Exiting plotting routine for KyotoGHG-AR4-total-excl-LU-change-since-1990-in-1990
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2000, dtype: float64)
upper outliers are: 
country
BOL     188.439306
GNQ    2801.477833
NER     164.248705
SYC     191.517857
TLS     165.700483
ZWE     155.434783
Name: 2000, dtype: float64
---
bins set to range(-156, 156, 12)
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2014, dtype: float64)
upper outliers are: 
country
GNQ    5909.852217
MDV     584.466019
QAT     538.297872
Name: 2014, dtype: float64
---
bins set to range(-444, 444, 37)
bins set to range(-210, 210, 14)
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2016, dtype: float64)
upper outliers are: 
country
NIU    192.957746
SGP    187.234043
Name: 201

## Below here is code for testing and debugging!

In [21]:
# read the data from file 
fname_in = os.path.join('proc-data', data_file_name)
data = pd.read_csv(fname_in)

data.columns

Index(['category', 'country', 'scenario', 'source', 'unit', 'variable', '1990',
       '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')

In [8]:
data_years

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,,,,,,
AFG,0.232670,0.203176,0.114439,0.101357,0.087184,0.079534,0.072379,0.065827,0.060963,0.048187,...,0.070288,0.091296,0.159742,0.247462,0.299967,0.417388,0.358342,0.321445,0.344954,0.358662
AGO,0.421478,0.407060,0.401747,0.432715,0.282488,0.770902,0.715148,0.490424,0.472127,0.576184,...,1.095625,1.195370,1.176502,1.223971,1.236674,1.246977,1.326897,1.250080,1.285267,1.360407
ALB,1.203735,1.700540,1.036849,1.025212,1.120807,1.113709,1.086595,1.121873,1.356420,1.457194,...,1.368548,1.494755,1.537613,1.691062,1.788796,2.036452,1.904084,1.952738,2.204894,1.966920
AND,7.888606,7.711175,7.573699,7.462564,7.323261,7.611590,8.017402,8.192516,8.621496,8.948268,...,7.445272,7.244536,7.118923,6.807795,6.856209,6.579026,6.611590,6.634649,6.576373,6.909016
ARE,24.943903,25.786462,24.824610,26.501707,27.740966,25.930856,20.731072,20.481406,26.813288,24.831318,...,28.424092,27.134047,27.559086,26.348767,23.577252,23.176775,24.043720,22.872972,28.111977,30.040521
ARG,3.391411,3.584994,3.654715,3.605884,3.732842,3.543382,3.783207,3.851094,3.945743,3.929266,...,4.322669,4.603427,4.655495,4.460849,4.608978,4.777122,4.703452,4.771988,4.769492,4.767634
ARM,6.161386,5.819840,7.261510,2.964529,3.008838,3.387890,2.345169,2.492750,2.126302,1.508571,...,1.466960,1.752438,1.980593,1.620171,1.623043,1.895269,2.186041,2.063239,2.064537,2.334630
ATG,4.962816,4.694905,4.588832,4.604078,4.461858,4.455372,4.588248,4.669471,4.696247,4.729259,...,5.603482,5.832722,5.893294,6.144410,6.232767,6.059403,6.117156,6.061907,6.139064,6.264824
AUS,16.371865,16.269225,16.355991,16.395779,16.496356,16.927828,17.130330,17.435953,18.001564,18.288695,...,19.150751,19.191600,19.023082,18.860078,18.444793,18.016175,17.877605,17.234878,16.826649,16.975107
